# Workforce Optimization with cuOpt Python API

This notebook demonstrates how to solve a workforce optimization problem using the cuOpt Python API. The problem involves assigning workers to shifts while minimizing total labor costs.

## Problem Description

We need to assign workers to shifts such that:
- Each shift has the required number of workers
- Workers can only be assigned to shifts they are available for
- Total labor cost is minimized

This is a classic assignment problem that can be formulated as a Mixed Integer Linear Program (MILP).


## Environment Setup

First, let's check if we have a GPU available and install necessary dependencies.


In [ ]:
# Check for GPU availability
!nvidia-smi


In [ ]:
# Install cuOpt if not already installed
# Uncomment the following line if running in Google Colab or similar environment
# !pip install --upgrade --extra-index-url https://pypi.nvidia.com --user cuopt-cu12 # For cuda 12
# !pip install --upgrade --extra-index-url https://pypi.nvidia.com --user cuopt-cu13 # For cuda 13


## Import Required Libraries


In [ ]:
import numpy as np
import pandas as pd
from cuopt.linear_programming.problem import Problem, VType, sense, LinearExpression, Constraint, CType
from cuopt.linear_programming.solver_settings import SolverSettings
import time


## Problem Data Setup

Define the shift requirements, worker pay rates, and availability constraints.


In [ ]:
# Number of workers required for each shift
shift_requirements = {
    "Mon1": 3,
    "Tue2": 2,
    "Wed3": 4,
    "Thu4": 4,
    "Fri5": 5,
    "Sat6": 6,
    "Sun7": 5,
    "Mon8": 2,
    "Tue9": 2,
    "Wed10": 3,
    "Thu11": 4,
    "Fri12": 6,
    "Sat13": 7,
    "Sun14": 5,
}

# Amount each worker is paid to work one shift
worker_pay = {
    "Amy": 10,
    "Bob": 12,
    "Cathy": 10,
    "Dan": 8,
    "Ed": 8,
    "Fred": 9,
    "Gu": 11,
}

# Worker availability (worker, shift) pairs
availability = [
    ("Amy", "Tue2"), ("Amy", "Wed3"), ("Amy", "Fri5"), ("Amy", "Sun7"),
    ("Amy", "Tue9"), ("Amy", "Wed10"), ("Amy", "Thu11"), ("Amy", "Fri12"),
    ("Amy", "Sat13"), ("Amy", "Sun14"),
    ("Bob", "Mon1"), ("Bob", "Tue2"), ("Bob", "Fri5"), ("Bob", "Sat6"),
    ("Bob", "Mon8"), ("Bob", "Thu11"), ("Bob", "Sat13"),
    ("Cathy", "Wed3"), ("Cathy", "Thu4"), ("Cathy", "Fri5"), ("Cathy", "Sun7"),
    ("Cathy", "Mon8"), ("Cathy", "Tue9"), ("Cathy", "Wed10"), ("Cathy", "Thu11"),
    ("Cathy", "Fri12"), ("Cathy", "Sat13"), ("Cathy", "Sun14"),
    ("Dan", "Tue2"), ("Dan", "Wed3"), ("Dan", "Fri5"), ("Dan", "Sat6"),
    ("Dan", "Mon8"), ("Dan", "Tue9"), ("Dan", "Wed10"), ("Dan", "Thu11"),
    ("Dan", "Fri12"), ("Dan", "Sat13"), ("Dan", "Sun14"),
    ("Ed", "Mon1"), ("Ed", "Tue2"), ("Ed", "Wed3"), ("Ed", "Thu4"),
    ("Ed", "Fri5"), ("Ed", "Sun7"), ("Ed", "Mon8"), ("Ed", "Tue9"),
    ("Ed", "Thu11"), ("Ed", "Sat13"), ("Ed", "Sun14"),
    ("Fred", "Mon1"), ("Fred", "Tue2"), ("Fred", "Wed3"), ("Fred", "Sat6"),
    ("Fred", "Mon8"), ("Fred", "Tue9"), ("Fred", "Fri12"), ("Fred", "Sat13"),
    ("Fred", "Sun14"),
    ("Gu", "Mon1"), ("Gu", "Tue2"), ("Gu", "Wed3"), ("Gu", "Fri5"),
    ("Gu", "Sat6"), ("Gu", "Sun7"), ("Gu", "Mon8"), ("Gu", "Tue9"),
    ("Gu", "Wed10"), ("Gu", "Thu11"), ("Gu", "Fri12"), ("Gu", "Sat13"),
    ("Gu", "Sun14"),
]

print(f"Number of shifts: {len(shift_requirements)}")
print(f"Number of workers: {len(worker_pay)}")
print(f"Number of available assignments: {len(availability)}")


In [ ]:
# Create DataFrames for better visualization
shifts_df = pd.DataFrame(list(shift_requirements.items()), columns=['Shift', 'Required Workers'])
workers_df = pd.DataFrame(list(worker_pay.items()), columns=['Worker', 'Pay per Shift'])

print("Shift Requirements:")
print(shifts_df)
print("\nWorker Pay Rates:")
print(workers_df)


## Problem Formulation

Now we'll create the optimization problem using the cuOpt Python API as a MILP. The problem has:
- **Variables**: Binary variables for each (worker, shift) assignment
- **Objective**: Minimize total labor cost
- **Constraints**: Meet shift requirements and respect worker availability


In [ ]:
# Create the optimization problem
problem = Problem("workforce_optimization")

# Add binary decision variables for each available (worker, shift) assignment
assignment_vars = {}
for worker, shift in availability:
    var_name = f"{worker}_{shift}"
    var = problem.addVariable(name=var_name, vtype=VType.BINARY, lb=0.0, ub=1.0)
    assignment_vars[(worker, shift)] = var

print(f"Created {len(assignment_vars)} binary decision variables")
print(f"Sample variables: {list(assignment_vars.keys())[:5]}")


In [ ]:
# Create objective function: minimize total labor cost
objective_expr = LinearExpression([], [], 0.0)

for (worker, shift), var in assignment_vars.items():
    cost = worker_pay[worker]
    if cost != 0:  # Only include non-zero coefficients
        objective_expr += var * cost

# Set objective function: minimize total cost
problem.setObjective(objective_expr, sense.MINIMIZE)
print("Objective function set: minimize total labor cost")


In [ ]:
# Add constraints: assign exactly the required number of workers to each shift
constraint_names = []

for shift, required_count in shift_requirements.items():
    # Find all workers available for this shift
    shift_assignments = []
    for (worker, shift_name), var in assignment_vars.items():
        if shift_name == shift:
            shift_assignments.append(var)
    
    if shift_assignments:
        # Create constraint: sum of assignments for this shift = required_count
        shift_expr = LinearExpression([], [], 0.0)
        for var in shift_assignments:
            shift_expr += var
        
        constraint = problem.addConstraint(shift_expr == required_count, name=f"shift_{shift}")
        constraint_names.append(f"shift_{shift}")
    else:
        print(f"Warning: No workers available for shift {shift}")

print(f"Added {len(constraint_names)} shift requirement constraints")
print(f"Sample constraints: {constraint_names[:5]}")


## Solver Configuration and Solution

Configure the solver settings and solve the optimization problem.


In [ ]:
# Configure solver settings
settings = SolverSettings()
settings.set_parameter("time_limit", 60.0)  # 60 second time limit
settings.set_parameter("log_to_console", True)  # Enable solver logging
settings.set_parameter("method", 0)  # Use default method

print("Solver configured with 60-second time limit")


In [ ]:
# Solve the problem
print("Solving workforce optimization problem...")
print(f"Problem type: {'MIP' if problem.IsMIP else 'LP'}")
print(f"Number of variables: {problem.NumVariables}")
print(f"Number of constraints: {problem.NumConstraints}")

start_time = time.time()
problem.solve(settings)
solve_time = time.time() - start_time

print(f"\nSolve completed in {solve_time:.3f} seconds")
print(f"Solver status: {problem.Status.name}")
print(f"Objective value: ${problem.ObjValue:.2f}")


## Solution Analysis

Let's analyze the optimal solution and create visualizations.


In [ ]:
def print_solution():
    """Print the optimal solution in a readable format"""
    if problem.Status.name == "Optimal" or problem.Status.name == "FeasibleFound":
        print(f"\nOptimal Solution Found!")
        print(f"Total Labor Cost: ${problem.ObjValue:.2f}")
        print("\nShift Assignments:")
        
        # Group assignments by shift
        shift_assignments = {}
        for (worker, shift), var in assignment_vars.items():
            if var.getValue() > 0.5:  # Binary variable is 1
                if shift not in shift_assignments:
                    shift_assignments[shift] = []
                shift_assignments[shift].append(worker)
        
        # Display assignments by shift
        for shift in sorted(shift_assignments.keys()):
            workers = shift_assignments[shift]
            required = shift_requirements[shift]
            total_cost = sum(worker_pay[w] for w in workers)
            print(f"  {shift}: {workers} (Required: {required}, Assigned: {len(workers)}, Cost: ${total_cost})")
        
        # Display assignments by worker
        print("\nWorker Assignments:")
        worker_assignments = {}
        for (worker, shift), var in assignment_vars.items():
            if var.getValue() > 0.5:
                if worker not in worker_assignments:
                    worker_assignments[worker] = []
                worker_assignments[worker].append(shift)
        
        for worker in sorted(worker_assignments.keys()):
            shifts = worker_assignments[worker]
            total_cost = len(shifts) * worker_pay[worker]
            print(f"  {worker}: {shifts} ({len(shifts)} shifts, ${total_cost})")
            
        return shift_assignments, worker_assignments
    else:
        print(f"No optimal solution found. Status: {problem.Status.name}")
        return None, None

shift_assignments, worker_assignments = print_solution()


In [ ]:
# Create a summary table of the solution
if shift_assignments:
    solution_data = []
    for shift in sorted(shift_assignments.keys()):
        workers = shift_assignments[shift]
        required = shift_requirements[shift]
        assigned = len(workers)
        total_cost = sum(worker_pay[w] for w in workers)
        
        solution_data.append({
            'Shift': shift,
            'Required': required,
            'Assigned': assigned,
            'Workers': ', '.join(workers),
            'Cost': f"${total_cost}"
        })
    
    solution_df = pd.DataFrame(solution_data)
    print("\nSolution Summary:")
    print(solution_df.to_string(index=False))


## Adding Additional Constraints

Now let's demonstrate how to add additional constraints to the existing model. We'll add a constraint to limit the maximum number of shifts per worker.


In [ ]:
# Add constraint: each worker can work at most 4 shifts per week
max_shifts_per_worker = 4

for worker in worker_pay.keys():
    # Find all shifts this worker is available for
    worker_shifts = []
    for (w, shift), var in assignment_vars.items():
        if w == worker:
            worker_shifts.append(var)
    
    if worker_shifts:
        # Create constraint: sum of shifts for this worker <= max_shifts_per_worker
        worker_expr = LinearExpression([], [], 0.0)
        for var in worker_shifts:
            worker_expr += var
        
        constraint = problem.addConstraint(worker_expr <= max_shifts_per_worker, 
                                        name=f"max_shifts_{worker}")

print(f"Added maximum shift constraints (max {max_shifts_per_worker} shifts per worker)")


In [ ]:
# Solve the problem again with the new constraints
print("\nSolving with maximum shift constraints...")
print(f"Problem now has {problem.NumVariables} variables and {problem.NumConstraints} constraints")

start_time = time.time()
problem.solve(settings)
solve_time = time.time() - start_time

print(f"\nSolve completed in {solve_time:.3f} seconds")
print(f"Solver status: {problem.Status.name}")
print(f"Objective value: ${problem.ObjValue:.2f}")


In [ ]:
# Display the new solution
shift_assignments_new, worker_assignments_new = print_solution()


## Solution Comparison

Let's compare the solutions before and after adding the maximum shift constraints.


In [ ]:
if worker_assignments and worker_assignments_new:
    print("\nWorker Assignment Comparison:")
    print("Worker\t\tBefore\t\tAfter")
    print("-" * 50)
    
    all_workers = set(worker_assignments.keys()) | set(worker_assignments_new.keys())
    for worker in sorted(all_workers):
        before_shifts = len(worker_assignments.get(worker, []))
        after_shifts = len(worker_assignments_new.get(worker, []))
        print(f"{worker}\t\t{before_shifts}\t\t{after_shifts}")
    
    # Check constraint violations
    print("\nConstraint Violations Check:")
    violations = 0
    for worker, shifts in worker_assignments_new.items():
        if len(shifts) > max_shifts_per_worker:
            print(f"  {worker}: {len(shifts)} shifts (exceeds limit of {max_shifts_per_worker})")
            violations += 1
    
    if violations == 0:
        print("  No constraint violations found.")
    else:
        print(f"  Found {violations} constraint violations.")


## Conclusion

This notebook demonstrated how to:

1. **Formulate a workforce optimization problem** using the cuOpt Python API
2. **Set up binary decision variables** for worker-shift assignments
3. **Define an objective function** to minimize total labor cost
4. **Add shift requirement constraints** to ensure proper staffing
5. **Solve the optimization problem** using cuOpt's high-performance solver
6. **Add additional constraints** to limit worker shifts
7. **Analyze and compare solutions** before and after constraint modifications

The cuOpt Python API provides a clean, intuitive interface for building and solving optimization problems, making it easy to model complex real-world scenarios like workforce scheduling.

### Key Benefits of cuOpt:
- **High Performance**: GPU-accelerated solving for large-scale problems
- **Easy to Use**: Intuitive Python API similar to other optimization libraries
- **Flexible**: Support for both LP and MIP problems
- **Scalable**: Handles problems with thousands of variables and constraints efficiently

### Problem Extensions:
This basic workforce optimization model can be extended with additional constraints such as:
- Minimum rest time between shifts
- Skill requirements for specific shifts
- Overtime cost considerations
- Worker preferences and fairness constraints
- Multi-week scheduling with carryover constraints


## License

SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: MIT
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
